In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np


In [2]:
# Charger la DataFrame avec les colonnes spécifiques
data = pd.read_csv("dataset_rempli_specific_rep_entreprise.csv")
#data.head(74745)
data = data.loc[:, ['Société', "RéP-l'entreprise"]]

In [3]:
datatwo = pd.read_csv("projet 2.csv")
datatwo = datatwo.loc[:, ['Contesté par le consommateur']]

C:\Users\chabc\AppData\Local\Temp\ipykernel_37512\3947249155.py:1: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  datatwo = pd.read_csv("projet 2.csv")


In [4]:
# Concaténer les colonnes
nouvelle_data = pd.concat([data, datatwo], axis=1)

In [5]:
nouvelle_data.head(7896)

,Société,RéP-l'entreprise,Contesté par le consommateur
0,Wells Fargo & Company,Company chooses not to provide a public response,No
1,Wells Fargo & Company,Company chooses not to provide a public response,No
2,Santander Bank US,Company has responded to the consumer and the ...,No
3,Wells Fargo & Company,Company chooses not to provide a public response,No
4,Franklin Credit Management,Company believes it acted appropriately as aut...,No
...,...,...,...
7891,"PayPal Holdings, Inc.",Company believes it acted appropriately as aut...,No
7892,Bank of America,Company chooses not to provide a public response,No
7893,"Servis One, Inc.",Company believes it acted appropriately as aut...,No
7894,Equifax,Company has responded to the consumer and the ...,No


In [6]:
# Ajouter une colonne "id_Contesté par consommateur" en fonction de la valeur de "Contesté par consommateur"
id_mapping = {'Yes': 1, 'No': 2}  # Ajoutez les autres valeurs si nécessaire
nouvelle_data['id_Contesté par le consommateur'] = nouvelle_data['Contesté par le consommateur'].map(id_mapping)
nouvelle_data.head(77896)

,Société,RéP-l'entreprise,Contesté par le consommateur,id_Contesté par le consommateur
0,Wells Fargo & Company,Company chooses not to provide a public response,No,2.0
1,Wells Fargo & Company,Company chooses not to provide a public response,No,2.0
2,Santander Bank US,Company has responded to the consumer and the ...,No,2.0
3,Wells Fargo & Company,Company chooses not to provide a public response,No,2.0
4,Franklin Credit Management,Company believes it acted appropriately as aut...,No,2.0
...,...,...,...,...
77891,Equifax,Company has responded to the consumer and the ...,Yes,1.0
77892,"SunTrust Banks, Inc.",Company chooses not to provide a public response,No,2.0
77893,EverBank,Company believes it acted appropriately as aut...,No,2.0
77894,The Huntington National Bank,Company chooses not to provide a public response,No,2.0


In [7]:
df_non_nan = nouvelle_data[nouvelle_data['Contesté par le consommateur'].isnull()]

In [8]:
df_non_nan.head()

,Société,RéP-l'entreprise,Contesté par le consommateur,id_Contesté par le consommateur
13,"Asset Management Professionals, LLC",Company chooses not to provide a public response,NaN,NaN
35,Land/Home Financial Services,Company believes it acted appropriately as aut...,NaN,NaN
156,"Southwest Credit Systems, L.P.",Company believes it acted appropriately as aut...,NaN,NaN
191,Embrace Home Loans Inc,Company believes it acted appropriately as aut...,NaN,NaN
540,"Eisenburg,Whitman and Associates",Company believes it acted appropriately as aut...,NaN,NaN


In [9]:
X = df_non_nan[['Société', "RéP-l'entreprise"]]
y = df_non_nan['id_Contesté par le consommateur']

In [10]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# Encoder les labels 'Yes' et 'No' en valeurs numériques
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

In [12]:
# Effectuer l'encodage one-hot sur les colonnes catégoriques
#L'encodage one-hot est essentiel car de nombreux algorithmes de machine learning ne peuvent pas traiter directement les variables catégoriques
X_train_encoded = pd.get_dummies(X_train)
X_test_encoded = pd.get_dummies(X_test)

In [13]:
# Remplacer les valeurs manquantes dans les données de test
X_test_encoded.fillna(0, inplace=True)  # Remplacer les valeurs manquantes par 0

In [14]:
# Vérifier les colonnes dans les données de test après l'encodage one-hot
#garantit que les colonnes après l'encodage one-hot sont cohérentes entre les ensembles d'entraînement et de test,
missing_cols = set(X_train_encoded.columns) - set(X_test_encoded.columns)
for col in missing_cols:
    X_test_encoded[col] = 0  # Ajouter les colonnes manquantes avec des valeurs 0
X_test_encoded = X_test_encoded[X_train_encoded.columns]  # Réorganiser les colonnes

C:\Users\chabc\AppData\Local\Temp\ipykernel_37512\1477852619.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_encoded[col] = 0  # Ajouter les colonnes manquantes avec des valeurs 0
C:\Users\chabc\AppData\Local\Temp\ipykernel_37512\1477852619.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_test_encoded[col] = 0  # Ajouter les colonnes manquantes avec des valeurs 0
C:\Users\chabc\AppData\Local\Temp\ipykernel_37512\1477852619.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of ca

In [15]:
# Créer et entraîner le modèle RandomForestClassifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_encoded, y_train_encoded)

RandomForestClassifier(random_state=42)

In [16]:
# Prédire les valeurs pour les données de test
y_pred_encoded = clf.predict(X_test_encoded)

In [17]:
# Convertir les prédictions numériques en labels originaux 'Consent not provided', 'Consent provided', etc.
y_pred = label_encoder.inverse_transform(y_pred_encoded)

In [18]:
try:
    # Prédire les valeurs manquantes
    missing_data = nouvelle_data[nouvelle_data['id_Contesté par le consommateur'].isna()]
    X_missing = missing_data[['Société', "RéP-l'entreprise"]]
    predicted_values_encoded = clf.predict(X_missing)
    predicted_values = label_encoder.inverse_transform(predicted_values_encoded)
except Exception as e:
    print("Une erreur s'est produite lors de la prédiction des valeurs manquantes :", e)
    # En cas d'erreur, attribuer la valeur 3 à l'ID
    predicted_values = [2] * len(missing_data)

Une erreur s'est produite lors de la prédiction des valeurs manquantes : The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- RéP-l'entreprise
- Société
Feature names seen at fit time, yet now missing:
- RéP-l'entreprise_Company believes complaint caused principally by actions of third party outside the control or direction of the company
- RéP-l'entreprise_Company believes complaint is the result of an isolated error
- RéP-l'entreprise_Company believes complaint relates to a discontinued policy or procedure
- RéP-l'entreprise_Company believes complaint represents an opportunity for improvement to better serve consumers
- RéP-l'entreprise_Company believes it acted appropriately as authorized by contract or law
- ...



In [19]:
# Remplacer les valeurs manquantes dans le DataFrame d'origine
nouvelle_data.loc[nouvelle_data['id_Contesté par le consommateur'].isna(), 'id_Contesté par le consommateur'] = predicted_values

In [20]:
nouvelle_data.isnull().any()

Société                            False
RéP-l'entreprise                   False
Contesté par le consommateur        True
id_Contesté par le consommateur    False
dtype: bool

In [21]:
# Créer un mapping inverse
inverse_mapping = {v: k for k, v in id_mapping.items()}

# Remplacer les valeurs manquantes dans le DataFrame d'origine avec les valeurs du mapping inverse
nouvelle_data['Contesté par le consommateur'] = nouvelle_data['Contesté par le consommateur'].fillna(nouvelle_data['id_Contesté par le consommateur'].map(inverse_mapping))

# Vérifier s'il y a des valeurs manquantes
nouvelle_data.isnull().any()

Société                            False
RéP-l'entreprise                   False
Contesté par le consommateur       False
id_Contesté par le consommateur    False
dtype: bool

In [22]:
nouvelle_data = nouvelle_data.loc[: , ["Contesté par le consommateur"]]

In [23]:
# Sauvegarder le DataFrame rempli dans un fichier Excel
nouvelle_data.to_csv("dataset_rempli_specific_ContestéparLeCons.csv", index=False)

In [24]:
nouvelle_data.to_excel("dataset_rempli_specific_FContestéparLeCons.xlsx", index=False)